In [1]:
import numpy as np
import pandas as pd
import tensorflow_recommenders as tfrs
import tensorflow as tf
from typing import Dict, Text

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
import pathlib
import os

user_file = "\\Data\\Combined Data.csv"
caregiver_file = "\\Data\\Processed Caregiver Data.csv"

path = pathlib.Path().resolve()
parent_path = path.parent.absolute()

user_dataframe = pd.read_csv(str(parent_path) + user_file)
caregiver_dataframe = pd.read_csv(str(parent_path) + caregiver_file)



In [3]:
unique_masalah_user = np.unique(' '.join(user_dataframe['Tipe_Masalah']).split(' '))
unique_masalah_caregiver = np.unique(' '.join(user_dataframe['Caregiver_Tipe_Masalah']).split(' '))
np.delete(unique_masalah_caregiver, np.where(unique_masalah_caregiver == ''))
unique_masalah_caregiver = np.delete(unique_masalah_caregiver, 0)

unique_masalah_caregiver = unique_masalah_caregiver.tolist()

for index, col in enumerate(unique_masalah_caregiver):
        unique_masalah_caregiver[index] = 'Caregiver-'+col


In [4]:
def convert_categorical_data(df, col='Tipe_Masalah'):
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(col=='Tipe_Masalah'):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)

In [5]:
convert_categorical_data(user_dataframe)
convert_categorical_data(user_dataframe, col='Caregiver_Tipe_Masalah')
convert_categorical_data(caregiver_dataframe, col='Caregiver_Tipe_Masalah')


caregiver_dataframe.drop(["CAREGIVEN_ID"],1,inplace=True)
user_dataframe.drop(["USER_ID"], 1,inplace=True)
user_dataframe.drop(["CAREGIVER_ID"], 1,inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3644\1950857100.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  caregiver_dataframe.drop(["CAREGIVEN_ID"],1,inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_3644\1950857100.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_dataframe.drop(["USER_ID"], 1,inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_3644\1950857100.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_dataframe.drop(["CAREGIVER_ID"], 1,inplace=True)


In [6]:

user_dataframe.head()



,Gender,Age,Caregiver_Gender,Caregiver_Age,ADHD-Hiperaktif-dan-kurang-fokus,Depresi,Gangguan-kecemasan,Gangguan-makan,Gangguan-stres-pascatrauma,Skizofrenia,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Depresi,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma,Caregiver-Skizofrenia
0,Pria,33,Pria,19,1,0,0,0,0,0,1,0,0,0,0,0
1,Pria,33,Pria,35,1,0,0,0,0,0,1,0,1,0,1,0
2,Pria,29,Perempuan,33,0,0,0,1,1,0,0,0,0,1,1,0
3,Pria,29,Pria,38,0,0,0,1,1,0,1,0,0,1,0,0
4,Perempuan,32,Pria,35,1,0,1,0,0,0,1,0,1,0,1,0


In [7]:

caregiver_dataframe.head()

,Caregiver_Gender,Caregiver_Age,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Depresi,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma,Caregiver-Skizofrenia
0,Perempuan,28,1,0,0,1,1,0
1,Pria,33,0,1,0,0,0,1
2,Pria,19,1,0,0,0,0,0
3,Perempuan,18,0,0,0,1,0,1
4,Perempuan,36,0,0,1,0,0,1


In [8]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [9]:
batch_size = 5# A small batch sized is used for demonstration purposes
caregiver_ds = df_to_dataset(caregiver_dataframe, batch_size=batch_size)
user_ds = df_to_dataset(user_dataframe, batch_size=batch_size)

In [10]:
feature_columns = []

number_feature = ["Age"]
number_feature += unique_masalah_user.tolist()
print(unique_masalah_caregiver)
# numeric cols
for header in number_feature:
    feature_columns.append(feature_column.numeric_column(header))

age_feature = ["Age"]
for col in age_feature:
    age = feature_column.numeric_column(col)
    age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
    feature_columns.append(age_buckets)

['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']


In [11]:
# indicator_columns
indicator_column_names = ['Gender']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, user_dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)



In [12]:
for col in feature_columns:
    print(col)

NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='ADHD-Hiperaktif-dan-kurang-fokus', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Depresi', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Gangguan-kecemasan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Gangguan-makan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Gangguan-stres-pascatrauma', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Skizofrenia', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(17, 21, 25, 29, 33, 37, 41, 46))
IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='G

In [13]:
feature_layer_user= tf.keras.layers.DenseFeatures(feature_columns)

In [14]:
feature_columns = []

number_feature = ["Caregiver_Age"]
number_feature += unique_masalah_caregiver
print(unique_masalah_caregiver)
# numeric cols

for header in number_feature:
    feature_columns.append(feature_column.numeric_column(header))

age_feature = ["Caregiver_Age"]
for col in age_feature:
    age = feature_column.numeric_column(col)
    age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
    feature_columns.append(age_buckets)

col_name = 'Caregiver_Gender'
categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, user_dataframe[col_name].unique())
indicator_column = feature_column.indicator_column(categorical_column)
feature_columns.append(indicator_column)

['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']


In [15]:
feature_layer_caregiver= tf.keras.layers.DenseFeatures(feature_columns)

In [16]:
class RecomendModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    embedding_dimension = 32

    # Set up a model for representing movies.
    self.user_model = tf.keras.Sequential([
        feature_layer_user,
        layers.Dense(64),
        layers.Dense(32)
    ])

    # Set up a model for representing users.
    self.caregiver_model = tf.keras.Sequential([
        feature_layer_caregiver,
        layers.Dense(64),
        layers.Dense(32)
    ])

    # Set up a task to optimize the model and compute metrics.
    self.task = tfrs.tasks.Retrieval(
      metrics=tfrs.metrics.FactorizedTopK(
        candidates=caregiver_ds.batch(5).map(self.caregiver_model)
      )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    
    caregiver_features = ['Caregiver_Gender', 'Caregiver_Age',  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
    user_features = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
    
    
    # We pick out the user features and pass them into the user model.
    
    print("AAAAAAAAAAAAAAAAAA")
    user_embeddings = self.user_model({
        'Age':features['Age'],
        'Gender':features['Gender'],
        'ADHD-Hiperaktif-dan-kurang-fokus':features['ADHD-Hiperaktif-dan-kurang-fokus'],
        'Depresi':features['Depresi'],
        'Gangguan-kecemasan':features['Gangguan-kecemasan'],
        'Gangguan-makan':features['Gangguan-makan'],
        'Gangguan-stres-pascatrauma':features['Gangguan-stres-pascatrauma'],
        'Skizofrenia':features['Skizofrenia']
    })
    print("BBBBBBBBBBBBBB")
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.caregiver_model({
        'Caregiver_Age':features['Caregiver_Age'],
        'Caregiver_Gender':features['Caregiver_Gender'],
        'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus':features['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus'],
        'Caregiver-Depresi':features['Caregiver-Depresi'],
        'Caregiver-Gangguan-kecemasan':features['Caregiver-Gangguan-kecemasan'],
        'Caregiver-Gangguan-makan':features['Caregiver-Gangguan-makan'],
        'Caregiver-Gangguan-stres-pascatrauma':features['Caregiver-Gangguan-stres-pascatrauma'],
        'Caregiver-Skizofrenia':features['Caregiver-Skizofrenia']
        
    })
    

    # The task computes the loss and the metrics.

    return self.task(user_embeddings, positive_movie_embeddings, compute_metrics=not training)

In [17]:
caregiver_features = ['Caregiver_Gender', 'Caregiver_Age',  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
user_features = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']

In [18]:

for feature_batch in caregiver_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus'])

Every feature: ['Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
A batch of ages: tf.Tensor([0 0 1 0 1], shape=(5,), dtype=int32)


In [19]:
model = RecomendModel()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(user_ds,
          epochs=10)

Epoch 1/10
AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBB
AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBB
40/40 [==============================] - 2s 7ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 33.8152 - regularization_loss: 0.0000e+00 - total_loss: 33.8152
Epoch 2/10
40/40 [==============================] - 0s 7ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15.7279 - regularization_loss: 0.0000e+00 - total_loss: 15.7279
Epoch 3/10
40/40 [==============================] - 0s 7ms/step - factor

40/40 [==============================] - 0s 7ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 8.2609 - regularization_loss: 0.0000e+00 - total_loss: 8.2609


In [20]:
brute_force = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
test_ds = caregiver_ds.map(lambda x: {"Caregiver_Age": x["Caregiver_Age"]})
print(test_ds)

<MapDataset element_spec={'Caregiver_Age': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}>


In [21]:
brute_force.index_from_dataset(  
    caregiver_ds.map(model.caregiver_model)
)


In [22]:
test_ds.map(lambda Caregiver_Age: (Caregiver_Age, model.caregiver_model(Caregiver_Age)))

ValueError: in user code:

    File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_3644\3304006942.py", line 1, in None  *
        lambda Caregiver_Age: (Caregiver_Age, model.caregiver_model(Caregiver_Age))
    File "C:\Users\ASUS\anaconda3\envs\notebook\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "dense_features_1" (type DenseFeatures).
    
    Feature Caregiver-ADHD-Hiperaktif-dan-kurang-fokus is not in features dictionary.
    
    Call arguments received:
      • features={'Caregiver_Age': 'tf.Tensor(shape=(None,), dtype=int64)'}
      • cols_to_output_tensors=None
      • training=False


In [23]:
print(my_dict)

NameError: name 'my_dict' is not defined

In [24]:
test_ds = caregiver_ds.map(lambda x: {"Caregiver_Age": x["Caregiver_Age"]})
test_ds.batch(128).map(lambda Caregiver_Age: (Caregiver_Age, model.caregiver_model(Caregiver_Age)))

ValueError: in user code:

    File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_3644\3821734079.py", line 2, in None  *
        lambda Caregiver_Age: (Caregiver_Age, model.caregiver_model(Caregiver_Age))
    File "C:\Users\ASUS\anaconda3\envs\notebook\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "dense_features_1" (type DenseFeatures).
    
    Feature Caregiver-ADHD-Hiperaktif-dan-kurang-fokus is not in features dictionary.
    
    Call arguments received:
      • features={'Caregiver_Age': 'tf.Tensor(shape=(None, None), dtype=int64)'}
      • cols_to_output_tensors=None
      • training=False


In [25]:
a =['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
value = ['Pria', 32, 1, 0, 0, 0, 1, 1]
dictionary_Test = dict((el,np.array([value[ix]])) for ix,el in enumerate(a))
print(dictionary_Test)

{'Gender': array(['Pria'], dtype='<U4'), 'Age': array([32]), 'ADHD-Hiperaktif-dan-kurang-fokus': array([1]), 'Depresi': array([0]), 'Gangguan-kecemasan': array([0]), 'Gangguan-makan': array([0]), 'Gangguan-stres-pascatrauma': array([1]), 'Skizofrenia': array([1])}


In [27]:
# Get predictions for user 42.
_, titles = brute_force(dictionary_Test, k=10)

print(f"Top recommendations: {titles[0]}")

Top recommendations: [23 15 25 22 14  8 17 20 27  1]


In [ ]:
tensor = model.user_model(dictionary_Test)
tensor.numpy()

In [ ]:
processed_ds = caregiver_ds.batch(5).map(model.caregiver_model)


In [ ]:
user_ds